# MACD strategy - BSE

In [1]:
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# BSE index code for Federal Bank
# Federal back = BOM500469
# TCS = BOM532540
# Bata = BOM500043
# Suzlon = BOM532667
index_code = "BOM532540"
start_date = "01-05-2008"
end_date = "01-05-2019"

In [3]:
# Convert date string to datetime object
start_date = datetime.strptime(start_date, "%d-%m-%Y")
end_date = datetime.strptime(end_date, "%d-%m-%Y")

In [4]:
# Get the data for the mentioned entity from Quandl
url = f"https://www.quandl.com/api/v3/datasets/BSE/{index_code}?api_key=JVU4wxLxNHFtSRjCvezu"

response = requests.get(url=url).json()

In [5]:
# Create a dataframe from the data for computation
df = pd.DataFrame(columns=response["dataset"]["column_names"], data=response["dataset"]["data"])

In [6]:
# Write data to csv
# df.to_csv(index_code+".csv", index=False)

In [7]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'WAP', 'No. of Shares',
       'No. of Trades', 'Total Turnover', 'Deliverable Quantity',
       '% Deli. Qty to Traded Qty', 'Spread H-L', 'Spread C-O'],
      dtype='object')

In [8]:
# Select subset of the complete data for ease of computation
df = df[["Date", "Open", "High", "Low", "Close"]]

In [9]:
# Fast length
df["MA12"] = df["Close"].ewm(span=12, adjust=False).mean()
# Slow Length
df["MA26"] = df["Close"].ewm(span=26, adjust=False).mean()

In [10]:
df["MACD"] = df["MA26"] - df["MA12"]

In [11]:
# Signal Length
df["signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

In [12]:
# Step 1 for decision
df["move"] = df["MACD"]>df["signal"]
df["move"] = df["move"].apply(lambda x: float(x))

In [13]:
# Step 2 for decision
df["positions"] = df["move"].diff()

In [14]:
# Convert the date column to datetime object
df["Date"] = pd.to_datetime(df["Date"])

In [15]:
# Subset the data based on date
rev_df = df[(df["Date"]>start_date)&(df["Date"]<=end_date)]
rev_df.sort_values("Date", ascending=True, inplace=True)

<ipython-input-15-7b9c4e1d159c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_df.sort_values("Date", ascending=True, inplace=True)


In [16]:

df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'MA12', 'MA26', 'MACD',
       'signal', 'move', 'positions'],
      dtype='object')

In [17]:
def deal_stocks(df, initial_money=30000, max_buy=10, max_sell=10):
    """Function to buy or sell based on the decision column i.e. positions
    
    Arguments:
    df -> Input dataframe containing the stock data
    initial_money -> Amount of money we want to put
    max_buy -> Maximum amount of shares we want to buy
    max_sell -> Maximum amount of shares we want to sell
    """
    money = initial_money
    sell_days = []
    buy_days = []
    shares_count = 0
    total_profit = 0
    total_loss = 0
    previous_units = 0
    previous_value = 0

    def buy(day, money, shares_count):
        date = day["Date"]
        close = day["Close"]
        shares = money // day["Close"]

        if shares < 1:
            print(f"Date: {date}: Total balance: {money}, not enough money to buy {shares} shares with unit price of {close}")
        else:
            if shares > max_buy:
                units = max_buy
            else:
                units = shares

            money = money - (units*day["Close"])
            shares_count = shares_count + units
            print(f"Date {date}: Buy {units} units of shares at a price of {close}, total balance is {money}")
            buy_days.append([date, close])
        return money, shares_count, units, day["Close"]

    def sell(day, money, shares_count):
        date = day["Date"]
        close = day["Close"]
        if shares_count == 0:
            print(f"Date {date}: Cannot sell anything as available shares is 0")
        else:
            if shares_count > max_sell:
                units = max_sell
            else:
                units = shares_count
            money = money + (units*day["Close"])
            shares_count = shares_count - units
            print(f"Date {date}: Sell {units} units of shares at a price of {close}, total balance is {money}")
            sell_days.append([date, close])
        return money, shares_count, units, day["Close"]

    for index, row in df.iterrows():
        state = row["positions"]

        if row['Date'] >= pd.to_datetime(end_date):
            break
        if state == 1.0:
            money, shares_count, previous_units, previous_value = buy(row, money, shares_count)
        elif state == -1.0:
            money, shares_count, units, price = sell(row, money, shares_count)
            change = (price*units) - (previous_value*previous_units)
            if change >= 0:
                total_profit += change
            elif change < 0:
                total_loss += abs(change)
        
            print(f'Total profits: {total_profit}')
            print(f'Total loss: {total_loss}')

    total_gains = total_profit - total_loss
    return buy_days, sell_days, total_gains

In [18]:
# Execute the deals
buy_data, sell_data, total_gain = deal_stocks(rev_df)

Date 2008-05-23 00:00:00: Buy 10 units of shares at a price of 933.7, total balance is 20663.0
Date 2008-06-09 00:00:00: Sell 10 units of shares at a price of 915.65, total balance is 29819.5
Total profits: 0
Total loss: 180.5
Date 2008-06-10 00:00:00: Buy 10 units of shares at a price of 880.05, total balance is 21019.0
Date 2008-07-09 00:00:00: Sell 10 units of shares at a price of 875.65, total balance is 29775.5
Total profits: 0
Total loss: 224.5
Date 2008-09-01 00:00:00: Buy 10 units of shares at a price of 814.35, total balance is 21632.0
Date 2008-10-14 00:00:00: Sell 10 units of shares at a price of 592.15, total balance is 27553.5
Total profits: 0
Total loss: 2446.5
Date 2008-10-20 00:00:00: Buy 10 units of shares at a price of 497.15, total balance is 22582.0
Date 2008-10-22 00:00:00: Sell 10 units of shares at a price of 546.35, total balance is 28045.5
Total profits: 492.0
Total loss: 2446.5
Date 2008-10-27 00:00:00: Buy 10 units of shares at a price of 497.0, total balance

In [19]:
print(f"If invested starting from {start_date}, then total gain would have been {total_gain}")

If invested starting from 2008-05-01 00:00:00, then total gain would have been 6191.5


In [20]:
# Creating buying and selling data dataframes
buy_df = pd.DataFrame(columns=["Date", "Close"], data=buy_data)
sell_df = pd.DataFrame(columns=["Date", "Close"], data=sell_data)

In [21]:
# Plot all the relevant values
close_chart = go.Scatter(x=rev_df["Date"], y=rev_df["Close"], mode="lines")
buy_chart = go.Scatter(x=buy_df["Date"], y=buy_df["Close"], mode="markers", marker_symbol="triangle-up", marker=dict(size=20, color="red"))
sell_chart = go.Scatter(x=sell_df["Date"], y=sell_df["Close"], mode="markers", marker_symbol="triangle-down", marker=dict(size=20, color="green"))
fig = go.Figure(data=[close_chart, buy_chart, sell_chart])
fig.update_layout(width=1200)
fig.show()